# Dataset Information
* Dataset: Apple Stock Price Dataset from Kaggle

* Source: https://www.kaggle.com/datasets/prasoonkottarathil/apple-lifetime-stocks-dataset

* Description: Historical Apple stock prices with OHLC (Open, High, Low, Close) data from 1980-2021

* Format: CSV with columns: Date, Open, High, Low, Close, Adj Close, Volume

* Size: 10,000+ daily records, perfect for time series learning

* Alternative: Any stock dataset from Yahoo Finance or similar

# Part1: Dataset Preparation and Exploration


## Libraries


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

## Load and Explore the Time Series Data

In [ ]:
# TODO 1.1.1: Load the dataset
# Download from: https://www.kaggle.com/datasets/prasoonkottarathil/apple-lifetime-stocks-dataset
df = pd.read_csv('AAPL.csv')

# TODO 1.1.2: Explore the dataset structure
print("Dataset shape:", df.shape)

print("\nFirst 5 rows:")
print(df.head())

print("\nDataset info:")
print(df.info())

print("\nMissing values:")
print(df.isnull().sum())

# TODO 1.1.3: Convert Date column to datetime and set as index
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)
# TODO 1.1.4: Display basic statistics
print("\nDataset statistics:")
print(df.describe())

# TODO 1.1.5: Check date range
print(f"\nDate range: {df.index.min().date()} to {df.index.max().date()} ")


Dataset shape: (9823, 7)

First 5 rows:
         Date      Open      High       Low     Close  Adj Close       Volume
0  1980-12-12  0.513393  0.515625  0.513393  0.513393   0.407747  117258400.0
1  1980-12-15  0.488839  0.488839  0.486607  0.486607   0.386473   43971200.0
2  1980-12-16  0.453125  0.453125  0.450893  0.450893   0.358108   26432000.0
3  1980-12-17  0.462054  0.464286  0.462054  0.462054   0.366972   21610400.0
4  1980-12-18  0.475446  0.477679  0.475446  0.475446   0.377609   18362400.0

Dataset info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9823 entries, 0 to 9822
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       9823 non-null   object 
 1   Open       9822 non-null   float64
 2   High       9822 non-null   float64
 3   Low        9822 non-null   float64
 4   Close      9822 non-null   float64
 5   Adj Close  9822 non-null   float64
 6   Volume     9822 non-null   float64
dtypes: flo

## Visualize Time Series Patterns